In [10]:
import os
import time

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = os.getenv("MODEL_NAME")
RUNPOD_TOKEN = os.getenv("RUNPOD_TOKEN")
RUNPOD_URL = os.getenv("RUNPOD_URL")




In [2]:
MODEL_NAME, RUNPOD_TOKEN, RUNPOD_URL

('meta-llama/Llama-3.2-1B-Instruct',
 'rpa_0DUW3JIK2N4I51QZXNXRLHAUPFPNUF56I26FRW6O19e0zj',
 'https://api.runpod.ai/v2/gbh9r4uu19q04i/openai/v1')

In [3]:
client = OpenAI(
    api_key=RUNPOD_TOKEN,
    base_url=RUNPOD_URL
)

In [7]:
def get_chatbot_response(client: OpenAI, model_name, messages):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message['role'], "content": message['content']})
    
    response = client.chat.completions.create(
        model = model_name,
        messages = input_messages,
        temperature = 0,
        top_p = 0.8,
        max_tokens = 2000
    ).choices[0].message.content

    return response

In [11]:
messages = [{'role': 'user', 'content': "What's the capital of Span ?"}]

start_time = time.time()
response = get_chatbot_response(client=client, model_name=MODEL_NAME, messages=messages)
end_time = time.time()


print(f"Inference time: {end_time-start_time}s")
response

Inference time: 1.9564154148101807s


'I think there might be some confusion. Spain is a country, not a state, and it does not have a capital city. The capital of Spain is Madrid.'

In [12]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [13]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,MODEL_NAME,messages)
print(response)

The iPhone 16 introduces several exciting updates, including:

1. Larger displays:
   - 6.1-inch base model display
   - 6.7-inch display for the iPhone 16 Plus
   - Thinner bezels and a more durable Ceramic Shield

2. Performance improvements:
   - Powered by the new A18 chip (A18 Pro for the Pro models)
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

3. Camera upgrades:
   - Dual-camera setup with a 48MP main sensor (base iPhone 16)
   - 48MP Ultra Wide and 5x telephoto camera (Pro models)
   - "Camera Control" button for more flexible photography options

4. Advanced audio features:
   - "Audio Mix" using machine learning to separate background sounds from speech
   - Refined audio capture during video recording

5. Battery life:
   - Extended battery life, especially in the iPhone 16 Pro Max

6. USB-C charging and data transfer:
   - Faster charging and data transfer
   - Support for up to 2x faster video encoding

7. Starting prices

In [18]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""


In [20]:
from sentence_transformers import SentenceTransformer, util

data = ["iphone_16", "samsung_s23"]
user_prompt = "What's new in iphone 16?"

# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode data and prompt
doc_embeddings = model.encode(data, convert_to_tensor=True)
prompt_embedding = model.encode(user_prompt, convert_to_tensor=True)

# Calculate cosine similarities
cosine_scores = util.cos_sim(prompt_embedding, doc_embeddings)

# Find the best match
best_match_idx = cosine_scores.argmax().item()
best_match_doc = data[best_match_idx]
best_score = cosine_scores[0, best_match_idx].item()

print(f"User prompt: {user_prompt}")
print(f"Best match document: {best_match_doc}")
print(f"Similarity score: {best_score:.4f}")


User prompt: What's new in iphone 16?
Best match document: iphone_16
Similarity score: 0.6883


In [2]:
import json

a= str({
    "chain of thought": "Not allowed", # The user is asking for a specific type of food (pizza) but is not asking about the coffee shop.
    "decision": "not allowed", # The user is not asking about the coffee shop or its staff.
    "message": "Sorry, I can't help you with that. May I help you with your order ?"
})

a = json.loads(a)

a

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [4]:
import json

# JSON formatted string
json_string = '''
{
    "chain of thought": "Not allowed", # The user is asking for a specific type of food (pizza) but is not asking about the coffee shop.
    "decision": "not allowed", # The user is not asking about the coffee shop or its staff.
    "message": "Sorry, I can't help you with that. May I help you with your order ?"
}
'''

# Parse the JSON string into a Python dictionary
data = json.loads(json_string)

# Access the data
print(data)
print(data["chain of thought"])  # Output: Not allowed


JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 40 (char 42)